In [1]:
import os
import pandas as pd
import gspread
import plotly.express as px

SPREADSHEET_KEY=os.environ['SPREADSHEET_KEY']
EAST_WEST_BANK_BANK_STATEMENTS_WORKSHEET_NAME='east_west_bank_bank_statements'
EAST_WEST_BANK_BANK_STATEMENTS_CATEGORIZED_WORKSHEET_NAME='east_west_bank_bank_statements_categorized'

In [2]:
gc = gspread.service_account()
finance_tracker_db_spreadsheet = gc.open_by_key(SPREADSHEET_KEY)

In [3]:
east_west_bank_bank_statements_worksheet = finance_tracker_db_spreadsheet.worksheet(EAST_WEST_BANK_BANK_STATEMENTS_WORKSHEET_NAME)
east_west_bank_bank_statements_df = pd.DataFrame(east_west_bank_bank_statements_worksheet.get_all_records())

In [4]:
description_category_mapping = {
    'Deposit': 'Deposit',
    'Mobile Check DE': 'Deposit',
    'ARTECH LLC DIR DEP': 'Paycheck',
    'INSYNC STAFFING': 'Paycheck',
    'CPRIME INC DIRECT DEP': 'Paycheck',
    'CARDMEMBER SERV WEB PYMT': 'CC Payment',
    'DISCOVER E-PAYMENT': 'CC Payment',
    'VENMO': 'Venmo',
    'ZELLE': 'Zelle',
    'PAYPAL': 'PayPal',
    'ROBINHOOD': 'Robinhood',
    'GOLDMAN SACHS': 'Marcus',
    'Goldman Sachs': 'Marcus',
    'WYZANT ACH ACH ITEMS': 'Tutoring',
    'JHULIVE': 'Tuition',
    'JHU REFUND': 'Tuition',
    'IRS TREAS': 'IRS',
    'ST OF CA DMV': 'DMV',
    'Trade App': 'Financial',
    'AMERITRADE': 'Financial',
    'FID BKG SVC LLC': 'Financial',
    'Public.com': 'Financial',
    'TASTYWORKS': 'Financial',
    'FIRSTRADE': 'Financial',
    'WEBULL': 'Financial',
    'IG US LLC': 'Financial',
    'OANDA': 'Financial',
    'Trading.com': 'Financial',
    'Apple Cash': 'Financial',
    '99 RANCH MARKET': 'Grocery',
    'COSTCO': 'Grocery',
}
for k, v in description_category_mapping.items():
    east_west_bank_bank_statements_df.loc[
        east_west_bank_bank_statements_df['description'].str.contains(k),
        'category'
    ] = v
east_west_bank_bank_statements_df.loc[
    east_west_bank_bank_statements_df['category'].isna(),
    'category'
] = 'Other'

east_west_bank_bank_statements_df['amount'] = east_west_bank_bank_statements_df['amount'].apply(
    lambda x: float(x.replace('$', '').replace(',', ''))
)

In [5]:
east_west_bank_bank_statements_categorized_worksheet = finance_tracker_db_spreadsheet.worksheet(
    EAST_WEST_BANK_BANK_STATEMENTS_CATEGORIZED_WORKSHEET_NAME
)
east_west_bank_bank_statements_categorized_worksheet.update(
    [east_west_bank_bank_statements_df.columns.values.tolist()] + \
    east_west_bank_bank_statements_df.values.tolist()
)
east_west_bank_bank_statements_categorized_worksheet.format("C:C", {"numberFormat": {"type": "CURRENCY"}})
east_west_bank_bank_statements_categorized_worksheet.format("A:A", {"numberFormat": {"type": "DATE_TIME"}})

{'spreadsheetId': '1CAyyf2kr-pS7LNX1a_0ithw6niL3Js3K4ZEOlwDViZY',
 'replies': [{}]}

In [43]:
pd.set_option('display.max_rows', 1000)

In [6]:
east_west_bank_bank_statements_categorized_worksheet = finance_tracker_db_spreadsheet.worksheet(EAST_WEST_BANK_BANK_STATEMENTS_CATEGORIZED_WORKSHEET_NAME)
east_west_bank_bank_statements_categorized_worksheet_df = pd.DataFrame(east_west_bank_bank_statements_categorized_worksheet.get_all_records())

marcus_worksheet = finance_tracker_db_spreadsheet.worksheet('marcus_bank_statements')
marcus_worksheet_df = pd.DataFrame(marcus_worksheet.get_all_records())

discover_credit_card_statements_worksheet = finance_tracker_db_spreadsheet.worksheet('discover_credit_card_statements')
discover_credit_card_statements_worksheet_df = pd.DataFrame(discover_credit_card_statements_worksheet.get_all_records())

robinhood_brokerage_worksheet = finance_tracker_db_spreadsheet.worksheet('robinhood_brokerage_modified')
robinhood_brokerage_worksheet_df = pd.DataFrame(robinhood_brokerage_worksheet.get_all_records())

robinhood_traditional_ira_worksheet = finance_tracker_db_spreadsheet.worksheet('robinhood_traditional_ira_modified')
robinhood_traditional_ira_worksheet_df = pd.DataFrame(robinhood_traditional_ira_worksheet.get_all_records())

east_west_bank_credit_card_statements_worksheet = finance_tracker_db_spreadsheet.worksheet('east_west_bank_credit_card_statements')
east_west_bank_credit_card_statements_worksheet_df = pd.DataFrame(east_west_bank_credit_card_statements_worksheet.get_all_records())

In [7]:
east_west_bank_credit_card_statements_worksheet_df['float_amount'] = east_west_bank_credit_card_statements_worksheet_df['amount'] \
    .apply(lambda x: float(x.replace('$', '').replace(',', '')))

In [8]:
net_worth_worksheet = finance_tracker_db_spreadsheet.worksheet('net_worth')
net_worth_worksheet_df = pd.DataFrame(net_worth_worksheet.get_all_records())

In [9]:
net_worth_worksheet_df

,update_date,total_assets,total_liabilities,net_worth,latest_net_worth
0,05/27/2024,"$97,122.76","-$2,212.59","$94,910.17","$94,910.17"


In [90]:
net_worth_worksheet_df.tail(1)['total_assets'].values[0]
net_worth_worksheet_df.tail(1)['total_liabilities'].values[0]
net_worth_worksheet_df.tail(1)['net_worth'].values[0]

0    $94,663.51
Name: total_assets, dtype: object

In [108]:
2982.74+30478.36+45198.00+18463.66+2212.59

99335.35

In [10]:
east_west_bank_balance = round(east_west_bank_bank_statements_categorized_worksheet_df.loc[
    ~east_west_bank_bank_statements_categorized_worksheet_df['amount'].isna()
]['amount'].apply(lambda x: float(x.replace('$', '').replace(',', ''))).sum(), 2)

marcus_balance = round(float(marcus_worksheet_df['balance'].tail(1).values[0].replace('$', '').replace(',', '')), 2)

robinhood_brokerage_portfolio_value = robinhood_brokerage_worksheet_df['Latest Portfolio Value'].tail(1).values[0]

robinhood_traditional_ira_worksheet_value = robinhood_traditional_ira_worksheet_df['Latest Portfolio Value'].tail(1).values[0]

east_west_bank_credit_card_balance = round(east_west_bank_credit_card_statements_worksheet_df.loc[
    ~east_west_bank_credit_card_statements_worksheet_df['amount'].isna()
]['amount'].apply(lambda x: float(x.replace('$', '').replace(',', ''))).sum(), 2)

discover_credit_card_balance = round(discover_credit_card_statements_worksheet_df.loc[
    ~discover_credit_card_statements_worksheet_df['Amount'].isna()
]['Amount'].apply(lambda x: float(x.replace('$', '').replace(',', ''))).sum(), 2)

In [110]:
sum(east_west_bank_credit_card_statements_worksheet_df['float_amount'].isna())

0

In [112]:
from datetime import datetime

In [120]:
east_west_bank_bank_statements_categorized_worksheet = finance_tracker_db_spreadsheet.worksheet(EAST_WEST_BANK_BANK_STATEMENTS_CATEGORIZED_WORKSHEET_NAME)
east_west_bank_bank_statements_categorized_worksheet_df = pd.DataFrame(east_west_bank_bank_statements_categorized_worksheet.get_all_records())

# # st.title('East West Bank Checking')
# todays_date = datetime.now()
# # date_range = st.date_input('Select Date Range', value=(datetime(todays_date.year, 1, 1), todays_date))
# start, end = datetime(todays_date.year, 1, 1), todays_date

In [125]:
east_west_bank_bank_statements_categorized_worksheet_df['transaction_date'] = pd.to_datetime(east_west_bank_bank_statements_categorized_worksheet_df['transaction_date'])
# east_west_bank_bank_statements_categorized_worksheet_df.loc[
#     (east_west_bank_bank_statements_categorized_worksheet_df['transaction_date'] >= start) &
#     (east_west_bank_bank_statements_categorized_worksheet_df['transaction_date'] <= end)
# ]

In [134]:
east_west_bank_bank_statements_categorized_worksheet_df['float_amount'] = \
    east_west_bank_bank_statements_categorized_worksheet_df['amount'] \
    .apply(lambda x: float(x.replace('$', '').replace(',', '')))
east_west_bank_bank_statements_categorized_worksheet_df.loc[0, 'float_amount'] += EAST_WEST_BANK_BANK_STATEMENT_INITIAL_BALANCE

In [139]:
east_west_bank_bank_statements_categorized_worksheet_df['float_amount_cumsum'] = east_west_bank_bank_statements_categorized_worksheet_df['float_amount'].cumsum()

In [137]:
fig = px.bar(east_west_bank_bank_statements_categorized_worksheet_df, x='category', y='float_amount')
fig.show()

In [140]:
east_west_bank_bank_statements_categorized_worksheet_df

,transaction_date,description,amount,category,float_amount,float_amount_cumsum
0,2020-01-02,Deposit,$510.00,Deposit,2969.25,2969.25
1,2020-01-06,Preauth Debit VENMO PAYMENT 200106,-$160.00,Venmo,-160.00,2809.25
2,2020-01-06,Preauth Debit VENMO PAYMENT 200106,-$180.00,Venmo,-180.00,2629.25
3,2020-01-06,Preauth Debit VENMO PAYMENT 200106,-$240.00,Venmo,-240.00,2389.25
4,2020-01-06,Preauth Debit VENMO PAYMENT 200106,-$370.00,Venmo,-370.00,2019.25
...,...,...,...,...,...,...
621,2024-04-08,Pre-Auth Credit CPRIME INC DIRECT ...,"$5,839.26",Paycheck,5839.26,9474.65
622,2024-04-08,Preauth Debit GOLDMAN SACHS BA COLLECTI...,"-$6,300.00",Marcus,-6300.00,3174.65
623,2024-04-23,Pre-Auth Credit CPRIME INC DIRECT ...,"$3,235.25",Paycheck,3235.25,6409.90
624,2024-04-23,Preauth Debit GOLDMAN SACHS BA COLLECTI...,"-$3,000.00",Marcus,-3000.00,3409.90


In [141]:
fig = px.line(
    east_west_bank_bank_statements_categorized_worksheet_df,
    x="transaction_date",
    y="float_amount_cumsum",
    title='Line Plot of Cumulative Sum of Amount'
)
fig.show()

In [54]:
east_west_bank_bank_statements_categorized_worksheet_df['float_amount'] = \
    east_west_bank_bank_statements_categorized_worksheet_df['amount'] \
    .apply(lambda x: float(x.replace('$', '').replace(',', '')))

EAST_WEST_BANK_BANK_STATEMENT_INITIAL_BALANCE=2459.25
east_west_bank_bank_statements_categorized_worksheet_df.loc[0, 'float_amount'] += EAST_WEST_BANK_BANK_STATEMENT_INITIAL_BALANCE

east_west_bank_bank_statements_categorized_worksheet_df['balance'] = \
    east_west_bank_bank_statements_categorized_worksheet_df['float_amount'].cumsum()

marcus_worksheet_df['float_balance'] = marcus_worksheet_df['balance'].apply(lambda x: float(x.replace('$', '').replace(',', '')))

In [89]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np

In [97]:
robinhood_brokerage_worksheet_df['Portfolio Value'].replace('', np.nan, inplace=True)
robinhood_brokerage_worksheet_df['Portfolio Value'].ffill(inplace=True)

In [99]:
fig = make_subplots(rows=1, cols=2)

fig.add_trace(
    go.Scatter(
        name='East West Bank Checking',
        x=east_west_bank_bank_statements_categorized_worksheet_df['transaction_date'].values.tolist(),
        y=east_west_bank_bank_statements_categorized_worksheet_df['balance'].values.tolist()
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        name='Marcus Savings',
        x=marcus_worksheet_df['transaction_date'].values.tolist(),
        y=marcus_worksheet_df['float_balance'].values.tolist()
    ),
    row=1, col=2
)

fig.show()

# fig.add_trace(
#     go.Scatter(
#         name='Robinhood Brokerage',
#         x=robinhood_brokerage_worksheet_df['Activity Date'].values.tolist(),
#         y=robinhood_brokerage_worksheet_df['Portfolio Value'].values.tolist()
#     ),
#     row=1, col=1
# )

In [29]:
assets = pd.DataFrame({
    'East West Bank': [east_west_bank_balance],
    'Marcus': [marcus_balance],
    'Robinhood Brokerage': [robinhood_brokerage_portfolio_value],
    'Robinhood Traditional IRA': [robinhood_traditional_ira_worksheet_value]
}).T
assets.reset_index(inplace=True)

In [30]:
assets.columns = ['Source', 'Value']

In [33]:
# fig = px.pie(assets, values='Value', names='Source', title='Pie Chart of Assets')
# fig.show()

In [62]:
total_assets = round(
    east_west_bank_balance + 
    marcus_balance + 
    robinhood_brokerage_portfolio_value + 
    robinhood_traditional_ira_worksheet_value,
    2
)

total_liabilities = round(
    east_west_bank_credit_card_balance + 
    discover_credit_card_balance,
    2
)

net_worth = round(total_assets + total_liabilities, 2)

In [85]:
net_worth_df = pd.DataFrame({
    'update_date': ['05/27/2024'],
    'total_assets': [total_assets],
    'total_liabilities': [total_liabilities],
    'net_worth': [net_worth],
})
net_worth_df['latest_net_worth'] = net_worth_df \
    .sort_values(by='update_date', ascending=True).tail(1)['net_worth'].values[0]

In [86]:
net_worth_worksheet = finance_tracker_db_spreadsheet.worksheet(
    'net_worth'
)
net_worth_worksheet.update(
    [net_worth_df.columns.values.tolist()] + \
    net_worth_df.values.tolist()
)
net_worth_worksheet.format("A:A", {"numberFormat": {"type": "DATE_TIME"}})
net_worth_worksheet.format("B:E", {"numberFormat": {"type": "CURRENCY"}})

{'spreadsheetId': '1CAyyf2kr-pS7LNX1a_0ithw6niL3Js3K4ZEOlwDViZY',
 'replies': [{}]}

In [74]:
# finance_tracker_db_spreadsheet = gc.open_by_key(SPREADSHEET_KEY)
# east_west_bank_worksheet = finance_tracker_db_spreadsheet.worksheet('east_west_bank_bank_statements')
# east_west_bank_worksheet.update([total_transactions_df.columns.values.tolist()] + total_transactions_df.values.tolist())

In [ ]:
# gc = gspread.service_account()
# finance_tracker_db_spreadsheet = gc.open_by_key(SPREADSHEET_KEY)
# east_west_bank_worksheet = finance_tracker_db_spreadsheet.worksheet('east_west_bank_bank_statements')
# east_west_bank_worksheet.update([total_transactions_df.columns.values.tolist()] + total_transactions_df.values.tolist())
# east_west_bank_worksheet.format("C:C", {"numberFormat": {"type": "CURRENCY"}})
# east_west_bank_worksheet.format("A", {"numberFormat": {"type": "DATE_TIME"}})

In [7]:
# gc = gspread.service_account()
# finance_tracker_db_spreadsheet = gc.open_by_key(SPREADSHEET_KEY)
# marcus_worksheet = finance_tracker_db_spreadsheet.worksheet('marcus_bank_statements')
# marcus_worksheet.update([transactions_df.columns.values.tolist()] + transactions_df.values.tolist())
# marcus_worksheet.format("C:D", {"numberFormat": {"type": "CURRENCY"}})

In [8]:
# gc = gspread.service_account()
# finance_tracker_db_spreadsheet = gc.open_by_key(SPREADSHEET_KEY)
# east_west_bank_credit_card_statements_worksheet = finance_tracker_db_spreadsheet.worksheet('east_west_bank_credit_card_statements')
# east_west_bank_credit_card_statements_worksheet.update([total_transactions_df.columns.values.tolist()] + total_transactions_df.values.tolist())
# east_west_bank_credit_card_statements_worksheet.format("D:D", {"numberFormat": {"type": "CURRENCY"}})
# east_west_bank_credit_card_statements_worksheet.format("A:B", {"numberFormat": {"type": "DATE_TIME"}})

In [9]:
# gc = gspread.service_account()
# finance_tracker_db_spreadsheet = gc.open_by_key(SPREADSHEET_KEY)
# east_west_bank_credit_card_statements_worksheet = finance_tracker_db_spreadsheet.worksheet('discover_credit_card_statements')
# east_west_bank_credit_card_statements_worksheet.update([df.columns.values.tolist()] + df.values.tolist())
# east_west_bank_credit_card_statements_worksheet.format("D:D", {"numberFormat": {"type": "CURRENCY"}})
# east_west_bank_credit_card_statements_worksheet.format("A:B", {"numberFormat": {"type": "DATE_TIME"}})

In [4]:
523.49+30478.36+45198+18463.66
# 523.49+45198+18463.66
# 523.49+30478.36+18463.66

94663.51000000001